In [31]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta
import time

import os

import warnings
warnings.filterwarnings('ignore')

In [49]:
#A function get url, direct ,the information on reservation time and df and returns an updated df with the 100 additional hotels.
Unsuccessful_list  = []
def get_hotels(url , driver ,booking_df,Snapshot_date,checking_date_type,checkout_date_type):
    driver.get(url)
    time.sleep(1)
    hotel_name_list =[]
    page_number=1
    index = 1
    while(len(hotel_name_list) <100 and page_number<=5): #take 100 hotel
        time.sleep(2)
        text_box = driver.find_elements(by= By.CLASS_NAME , value='b978843432')
        for t_box in text_box:# run on full page
            if (len(hotel_name_list) >= 100):
                break
            try:
                hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Grade":None,"Num of Reviews":None,"Curr Price":None,"Original Price":None,"Distance from center":None,"Type of room":None,"Location grade":None ,"Extra Data":None}
                name = t_box.find_element(by= By.CLASS_NAME , value="e13098a59f").text
                name = name.split('\n')[0]
                if(name in hotel_name_list):#Avoid hotels that are advertised several times
                    continue   
                if (name in Unsuccessful_list):
                    continue
                hotel['Hotel Name'] = (name)
                hotel['Snapshot'] = Snapshot_date
                delta_TTT =  checking_date_type - Snapshot_date
                hotel['TTT'] = delta_TTT.days
                delta_LOS = checkout_date_type - checking_date_type
                hotel['LOS'] = delta_LOS.days
                score_detail = t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="review-score"]').text
                score_detail_list = score_detail.split('\n')
                hotel["Grade"] = (score_detail_list[0])
                hotel["Num of Reviews"] = (score_detail_list[2])
                hotel["Curr Price"] = (t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="price-and-discounted-price"]').text)
                hotel["Index"] = index
                index = index +1
                hotel["Extra Data"] = t_box.find_element(by= By.CLASS_NAME , value="d22a7c133b").text
                hotel["Distance from center"] =  t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="distance"]').text
                hotel["Type of room"] = t_box.find_element(by= By.CLASS_NAME , value="df597226dd").text
                
                try:
                    hotel["Location grade"] = t_box.find_element(by= By.CLASS_NAME , value="f9afbb0024").text 
                except:
                    hotel["Location grade"] = None
                #need to find good way to get Original Price
                '''try:
                    hotel["Original Price"] = t_box.find_element(by= By.CLASS_NAME, value="c5888af24f e729ed5ab6").text
                except:
                    hotel["Original Price"] = hotel["Curr Price"]'''
                
                booking_df = booking_df.append(hotel, ignore_index=True)
                hotel_name_list.append(name) 
            except:
                print("error " + name)
                Unsuccessful_list.append(name)
        # change page
        if( len(hotel_name_list)< 100): 
            button=driver.find_element(by =By.CSS_SELECTOR,value =f'[aria-label=" {page_number+1}"]')
            time.sleep(1.5)
            button.click() 
            page_number = page_number+1
    
    
    return booking_df
    

In [41]:
#Allows us to run on different computers without a problem

path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': 0 , 'yair':0 }
driver = webdriver.Chrome(path['roy'])

In [6]:
# Get today's date
now = datetime.now()
Snapshot_date =now
# Get end date 35 days from now
end_date = now + timedelta(days=35)

# Create a list of all dates between today and end date
date_list = []
while now <= end_date:
    date_list.append(now.strftime('%Y-%m-%d'))
    now += timedelta(days=1)


#Creates a list whose every member is a list of checking and checkout time

combinations_list =[]
for i in range(1,len(date_list)-5):
    for j in range(1,6):
        combinations = [date_list[i],date_list[i+j]]
        combinations_list.append(combinations)
        
print(combinations_list , len(combinations_list))        
    

[['2023-02-05', '2023-02-06'], ['2023-02-05', '2023-02-07'], ['2023-02-05', '2023-02-08'], ['2023-02-05', '2023-02-09'], ['2023-02-05', '2023-02-10'], ['2023-02-06', '2023-02-07'], ['2023-02-06', '2023-02-08'], ['2023-02-06', '2023-02-09'], ['2023-02-06', '2023-02-10'], ['2023-02-06', '2023-02-11'], ['2023-02-07', '2023-02-08'], ['2023-02-07', '2023-02-09'], ['2023-02-07', '2023-02-10'], ['2023-02-07', '2023-02-11'], ['2023-02-07', '2023-02-12'], ['2023-02-08', '2023-02-09'], ['2023-02-08', '2023-02-10'], ['2023-02-08', '2023-02-11'], ['2023-02-08', '2023-02-12'], ['2023-02-08', '2023-02-13'], ['2023-02-09', '2023-02-10'], ['2023-02-09', '2023-02-11'], ['2023-02-09', '2023-02-12'], ['2023-02-09', '2023-02-13'], ['2023-02-09', '2023-02-14'], ['2023-02-10', '2023-02-11'], ['2023-02-10', '2023-02-12'], ['2023-02-10', '2023-02-13'], ['2023-02-10', '2023-02-14'], ['2023-02-10', '2023-02-15'], ['2023-02-11', '2023-02-12'], ['2023-02-11', '2023-02-13'], ['2023-02-11', '2023-02-14'], ['2023-02

In [50]:
city = "new york"
numOfAdults=2
numOfChild=0
numOfRooms=1
booking_df = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Grade","Num of Reviews","Curr Price","Original Price","Distance from center","Type of room","Location grade", "Extra Data"])  
Unsuccessful_list = []
for comb in combinations_list:
    if comb != combinations_list[0]: # for easy test
        break
    checking_date_type = datetime.strptime(comb[0], '%Y-%m-%d')
    checkout_date_type = datetime.strptime(comb[1], '%Y-%m-%d')
    checking_date = comb[0].split('-')
    checkout_date = comb[1].split('-')
    checking_year = checking_date[0]
    checking_month = checking_date[1]
    checking_monthday =checking_date[2]
    checkout_year= checkout_date[0]
    checkout_month= checkout_date[1]
    checkout_monthday= checkout_date[2]
    url = f"https://www.booking.com/searchresults.html?ss={city}&checkin_year={checking_year}&checkin_month={checking_month}&checkin_monthday={checking_monthday}&checkout_year={checkout_year}&checkout_month={checkout_month}&checkout_monthday={checkout_monthday}&group_adults={numOfAdults}&group_children={numOfChild}&no_rooms={numOfRooms}"
    booking_df = get_hotels(url, driver , booking_df, Snapshot_date,checking_date_type,checkout_date_type)
    

In [51]:
booking_df.head(6500)

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Distance from center,Type of room,Location grade,Extra Data
0,2023-02-04 00:02:48.134854,1,Riu Plaza Manhattan Times Square,0,1,8.6,"6,051 reviews",₪ 470,None,1 km from center,Deluxe King,Location 9.5,Deluxe King\n1 king bed
1,2023-02-04 00:02:48.134854,2,Hyatt Place New York City/Times Square,0,1,7.6,"14,668 reviews",₪ 432,None,1.7 km from center,Deluxe King Room,None,Deluxe King Room\n1 king bed\nBreakfast includ...
2,2023-02-04 00:02:48.134854,3,Pod Times Square,0,1,8.1,"12,580 reviews",₪ 302,None,1.4 km from center,Standard Bunk Bed Room,Location 9.3,Standard Bunk Bed Room\n1 bunk bed\nFREE cance...
3,2023-02-04 00:02:48.134854,4,Courtyard by Marriott New York Manhattan/Centr...,0,1,8.2,"3,225 reviews",₪ 607,None,450 m from center,King or Queen Room with City View,Location 9.4,King or Queen Room with City View\n1 queen bed...
4,2023-02-04 00:02:48.134854,5,The Cloud One New York-Downtown,0,1,8.1,423 reviews,₪ 505,None,7 km from center,Standard Double Room with Two Double Beds,None,Standard Double Room with Two Double Beds\n2 f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-02-04 00:02:48.134854,96,"InterContinental New York Barclay Hotel, an IH...",0,1,8.0,907 reviews,₪ 675,None,1.5 km from center,Classic Queen Room,None,Classic Queen Room\n1 queen bed
96,2023-02-04 00:02:48.134854,97,DoubleTree by Hilton New York Times Square South,0,1,7.8,"4,293 reviews",₪ 424,None,1.8 km from center,King Room with Empire State Building View,None,King Room with Empire State Building View\n1 k...
97,2023-02-04 00:02:48.134854,98,Millennium Downtown New York,0,1,7.4,"3,566 reviews",₪ 394,None,6.8 km from center,Superior Queen Room,None,Superior Queen Room\n1 queen bed\nFREE cancell...
98,2023-02-04 00:02:48.134854,99,SpringHill Suites by Marriott New York Midtown...,0,1,8.1,49 reviews,₪ 539,None,3.1 km from center,"Studio, 1 King",Location 9.3,"Studio, 1 King\n1 king bed\nBreakfast included..."


In [13]:
booking_df.shape

(15000, 14)

In [14]:
booking_df.groupby(['TTT','LOS']).count()

Snapshot  Index  Hotel Name  Num of stars  Grade  Num of Reviews  \
TTT LOS                                                                     
0   1         100    100         100             0    100             100   
    2         100    100         100             0    100             100   
    3         100    100         100             0    100             100   
    4         100    100         100             0    100             100   
    5         100    100         100             0    100             100   
...           ...    ...         ...           ...    ...             ...   
29  1         100    100         100             0    100             100   
    2         100    100         100             0    100             100   
    3         100    100         100             0    100             100   
    4         100    100         100             0    100             100   
    5         100    100         100             0    100             100   

         Curr Price  Original Price  Percentage of discount (Boolean)  \
TTT LOS                                                                 
0   1           100               0                                 0   
    2           100               0                                 0   
    3           100               0                                 0   
    4           100               0                                 0   
    5           100               0                                 0   
...             ...             ...                               ...   
29  1           100               0                                 0   
    2           100               0                                 0   
    3           100               0                                 0   
    4           100               0                                 0   
    5           100               0                                 0   

         Distance from center  Type of room  Location grade  
TTT LOS                                                      
0   1                       0             0               0  
    2                       0             0               0  
    3                       0             0               0  
    4                       0             0               0  
    5                       0             0               0  
...                       ...           ...             ...  
29  1                       0             0               0  
    2                       0             0               0  
    3                       0             0               0  
    4                       0             0               0  
    5                       0             0               0  

[150 rows x 12 columns]

save the df in csv

In [32]:
file_name = "Booking_hotels.csv"
if os.path.exists(file_name):
    print("exists")
    booking_df.to_csv(file_name,mode = 'a' , index= False ,header=False)
else:
    print("new")
    booking_df.to_csv(file_name,mode = 'a' , index= False ,header=True)

new


here we try find more thing

In [ ]:
path ={'roy' :'C:\Windows\chromdriver.exe' , 'yarin': 0 , 'yair':0 }
driver = webdriver.Chrome(path['roy'])


city = "new york"
checking_year = 2023
checking_month = 2
checking_monthday =7
checkout_year=2023
checkout_month=2
checkout_monthday=9
numOfAdults=2
numOfChild=0
numOfRooms=1
url = f"https://www.booking.com/searchresults.html?ss={city}&checkin_year={checking_year}&checkin_month={checking_month}&checkin_monthday={checking_monthday}&checkout_year={checkout_year}&checkout_month={checkout_month}&checkout_monthday={checkout_monthday}&group_adults={numOfAdults}&group_children={numOfChild}&no_rooms={numOfRooms}"
driver.get(url)



In [ ]:
booking_df_test = pd.DataFrame(columns=['Snapshot','Index','Hotel Name','TTT' ,'LOS' ,"Num of stars","Grade","Num of Reviews","Curr Price","Original Price","Percentage of discount (Boolean)","Distance from center","Type of room","Location grade"])  

hotel_name_list =[]
page_number=1
index = 1
while(len(hotel_name_list) <100 and page_number<6): #take 100 hotel
    time.sleep(2)
    text_box = driver.find_elements(by= By.CLASS_NAME , value='b978843432')
    for t_box in text_box:# run on full page
        try:
            hotel = {'Snapshot' : None ,'Index':None,'Hotel Name':None,'TTT':None,'LOS':None ,"Num of stars":None,"Grade":None,"Num of Reviews":None,"Curr Price":None,"Original Price":None,"Percentage of discount (Boolean)":None,"Distance from center":None,"Type of room":None,"Location grade":None}
            name = t_box.find_element(by= By.CLASS_NAME , value="e13098a59f").text
            name = name.split('\n')[0]  
            hotel['Hotel Name'] = (name)
            score_detail = t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="review-score"]').text
            score_detail_list = score_detail.split('\n')
            hotel["Grade"] = (score_detail_list[0])
            hotel["Num of Reviews"] = (score_detail_list[2])
            hotel["Curr Price"] = (t_box.find_element(by =By.CSS_SELECTOR,value ='[data-testid="price-and-discounted-price"]').text)
            hotel["Index"] = index
            index = index +1
            booking_df_test.append(hotel, ignore_index=True)
            hotel_name_list.append(name) 
        except:
            print("error")
    # change page        
    button=driver.find_element(by =By.CSS_SELECTOR,value =f'[aria-label=" {page_number+1}"]')
    time.sleep(1.5)
    button.click() 
    page_number = page_number+1
    
    

In [ ]:
booking_df_test.head(200)

In [ ]:
booking_df_test.shape